# Examen ETL: SPARK 09/02
## Práctica 02 ETL    
27/12/2018

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen o en su defecto por email.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: ¿ En que se diferencian las acciones de las transformaciones en SPARK? ¿Podrías ponerme un par de ejemplo de cada una de ellas y describir que hacen, para que sirve?

Las transformaciones crean un nuevo dataset sobre uno ya existente y las acciones que devuelven valores al driver después de hacer una computación en un dataset.

- Map(transformación): Pasa cada elemento de un dataset a través de una función especificada y devuelve un RDD nuevo con los valores especificados por el usuario.
- Reduce (acción): Agrega todos los elementos del RDD usando una función y devuelve el resultado final al driver.
- Filter (transformación): Filtra los elementos del dataset en función de las restricciones especificadas.
- Collect (acción): Devuelve todos los elementos del dataset especificado.

## Segunda tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
from pyspark import SQLContext
sqlContext = SQLContext(sc)

In [28]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt

In [29]:
datos = "./DataSetPartidos.txt"
raw_data = sc.textFile(datos)

In [30]:
raw_data.take(1)

['1::1970-71::1::Athletic Club::Barcelona::1::1::12/09/1970::21942000.0']

In [31]:
raw_data.count()

34518

In [32]:
datos_tratados = raw_data.map(lambda x: x.split('::'))
datos_tratados.take(1)

[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

In [33]:
DatosTratadosDF = datos_tratados.map(lambda x: Row(Id = x[0],
                                                    Temporada = x[1],
                                                    Jornada = int(x[2]),
                                                    Local = x[3],
                                                    Visitante = x[4],
                                                    GolLocal = int(x[5]),
                                                    GolVisitante = int(x[6]),
                                                    Date = dt.datetime.strptime(x[7], '%d/%m/%Y').strftime('%d/%m/%Y'),
                                                    Unknown2 = x[8]))

df_interact = sqlContext.createDataFrame(DatosTratadosDF)
df_interact.registerTempTable('interactions')

In [34]:
df_interact.show()

+----------+--------+------------+---+-------+------------------+---------+----------+------------------+
|      Date|GolLocal|GolVisitante| Id|Jornada|             Local|Temporada|  Unknown2|         Visitante|
+----------+--------+------------+---+-------+------------------+---------+----------+------------------+
|12/09/1970|       1|           1|  1|      1|     Athletic Club|  1970-71|21942000.0|         Barcelona|
|12/09/1970|       1|           1|  2|      1|        Las Palmas|  1970-71|21942000.0|Atletico de Madrid|
|12/09/1970|       2|           0|  3|      1|       Real Madrid|  1970-71|21942000.0|          Valencia|
|13/09/1970|       2|           0|  4|      1|     Celta de Vigo|  1970-71|22028400.0| Sporting de Gijon|
|13/09/1970|       1|           1|  5|      1|             Elche|  1970-71|22028400.0|           Granada|
|13/09/1970|       0|           1|  6|      1|          Espanyol|  1970-71|22028400.0|           Sevilla|
|13/09/1970|       0|           0|  7|      1|

## Tercera pregunta: Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

A través de RDD

In [35]:
from operator import add

sporting_local = datos_tratados.filter(lambda x: x[3] == 'Sporting de Gijon').map(lambda x: int(x[5])).reduce(add)
sporting_visitante = datos_tratados.filter(lambda x: x[4] == 'Sporting de Gijon').map(lambda x: int(x[6])).reduce(add)

sporting_local + sporting_visitante

2042

Resolución del apartado 3 a través de DataFrame

In [36]:
como_local = df_interact.select('Local', 'GolLocal').filter(df_interact.Local == 'Sporting de Gijon')
como_local_acum = como_local.groupBy('Local').sum()

como_visitante = df_interact.select('Visitante', 'GolVisitante').filter(df_interact.Visitante == 'Sporting de Gijon')
como_visitante_acum = como_visitante.groupBy('Visitante').sum()

como_local_acum.take(1)[0][1] + como_visitante_acum.take(1)[0][1]

2042

## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

In [37]:
top_goles = df_interact.select('Temporada', 'GolLocal', 'GolVisitante')
top_goles = top_goles.withColumn('Total Goles', top_goles.GolLocal + top_goles.GolVisitante)
top_goles.groupBy('Temporada').sum().sort(desc('sum(Total Goles)')).show(1)

+---------+-------------+-----------------+----------------+
|Temporada|sum(GolLocal)|sum(GolVisitante)|sum(Total Goles)|
+---------+-------------+-----------------+----------------+
|  2012-13|         1363|              931|            2294|
+---------+-------------+-----------------+----------------+
only showing top 1 row



In [ ]:
data_clean.map(lambda x: (x[1], int(x[5])+int(x[6]))).reduceByKey(add).sortBy(lambda x: x[1], ascending = False).take(10)

## Quinta pregunta: ¿Cúal es el equipo que tiene el record de más goles como local? ¿ y cómo visitante?

Record como local

In [38]:
record_goles_local = df_interact.select('Local', 'GolLocal')
record_goles_local.groupBy('Local').sum().sort(desc('sum(GolLocal)')).show(5)

+------------------+-------------+
|             Local|sum(GolLocal)|
+------------------+-------------+
|       Real Madrid|         2054|
|         Barcelona|         2043|
|Atletico de Madrid|         1566|
|          Valencia|         1522|
|     Athletic Club|         1424|
+------------------+-------------+
only showing top 5 rows



Record como visitante

In [39]:
record_goles_visitante = df_interact.select('Visitante', 'GolVisitante')
record_goles_visitante.groupBy('Visitante').sum().sort(desc('sum(GolVisitante)')).show(5)

+------------------+-----------------+
|         Visitante|sum(GolVisitante)|
+------------------+-----------------+
|       Real Madrid|             1296|
|         Barcelona|             1296|
|Atletico de Madrid|              988|
|          Valencia|              919|
|             Betis|              853|
+------------------+-----------------+
only showing top 5 rows



In [ ]:
Local = data_clean.map(lambda x: (x[3], int(x[5]))).reduceByKey(add).sortBy(lambda x: x[1],ascending = False).take(1)
Local

In [ ]:
visitante = data_clean.map(lambda x: (x[4], int(x[6]))).reduceByKey(add).sortBy(lambda x: x[1], ascending = False).take(3)
visitante

## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

In [40]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf

def getDecade(x):
    res = (x // 10) * 10
    return res

split_date = split(df_interact.Date, '/')

#Creamos la columna del año
df_interact_date_split = df_interact.withColumn('Year', split_date.getItem(2)).select('GolLocal', 'GolVisitante', 'Year')

#Cambiamos el tipo de la columna a INT
df_interact_date_split = df_interact_date_split.withColumn('Year', df_interact_date_split.Year.cast('int'))

#Creamos nuevo DF con el total de goles por fila
df_dt_decades = df_interact_date_split\
                .withColumn('Total_Goles', df_interact_date_split.GolLocal + df_interact_date_split.GolVisitante)

    #funcion definida por el usuario
conv_to_decade = udf(getDecade, IntegerType())

#Aplicamos la funcion a la columna Year y la guardamos en Decade
df_dt_decades = df_dt_decades.withColumn('Decade',conv_to_decade(df_dt_decades['Year'])).select('Decade', 'Total_Goles')

df_dt_decades.groupBy('Decade').sum().sort(desc('sum(Total_Goles)'))\
            .select('Decade', col('sum(Total_Goles)').alias('Goles totales'))\
            .show()

+------+-------------+
|Decade|Goles totales|
+------+-------------+
|  2000|        20526|
|  1990|        19321|
|  1980|        17336|
|  1970|        15222|
|  2010|        12061|
+------+-------------+



##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

Primero vamos a obtener las medias siendo Local, para ello nos definimos una UDF que pasaremos a las columnas GolLocal y GolVisitante con el objetivo de determinar si ganó (1) o perdió(0), además de un contador de 'apariciones' que servirá para filtrar aquellos que han aparecido menos de 10 veces

In [41]:
import time

def getVictory(x, y):
    res = 0
    #Si el primero que le pasemos 
    #ha metido mas goles que el segundo
    if x > y:
        res = 1
        return res
    else:
        return res

#Registramos la funcion udf
get_victory = udf(getVictory, IntegerType())

#Preparamos el DataFrame completo
vic_mean = df_interact.select('Jornada', 'GolLocal', 'GolVisitante', 'Temporada', 'Local', 'Visitante', 'Temporada')
vic_mean = vic_mean.withColumn('LocalWon', get_victory(vic_mean.GolLocal, vic_mean.GolVisitante))
vic_mean = vic_mean.withColumn('VisWon', get_victory(vic_mean.GolVisitante, vic_mean.GolLocal))
vic_mean.show(5)

+-------+--------+------------+---------+-------------+------------------+---------+--------+------+
|Jornada|GolLocal|GolVisitante|Temporada|        Local|         Visitante|Temporada|LocalWon|VisWon|
+-------+--------+------------+---------+-------------+------------------+---------+--------+------+
|      1|       1|           1|  1970-71|Athletic Club|         Barcelona|  1970-71|       0|     0|
|      1|       1|           1|  1970-71|   Las Palmas|Atletico de Madrid|  1970-71|       0|     0|
|      1|       2|           0|  1970-71|  Real Madrid|          Valencia|  1970-71|       1|     0|
|      1|       2|           0|  1970-71|Celta de Vigo| Sporting de Gijon|  1970-71|       1|     0|
|      1|       1|           1|  1970-71|        Elche|           Granada|  1970-71|       0|     0|
+-------+--------+------------+---------+-------------+------------------+---------+--------+------+
only showing top 5 rows



In [42]:
#Obtenemos por parte del equipo Local
t0 = time.time()

mv_filter_local = vic_mean.select('Local', 'Temporada', 'LocalWon')\
                                 .withColumn('AparicionesTotales', lit(1))\
                                 .groupBy(['Local', 'Temporada'])\
                                 .sum()\
                                 .withColumn('Por_temporada', lit(1))\
                                 .groupBy('Local')\
                                 .sum()\
                                 .sort('Local')

mv_filter_local.filter(mv_filter_local['sum(Por_temporada)'] < 10)\
                .withColumn('Media_victorias', mv_filter_local['sum(sum(LocalWon))'] / mv_filter_local['sum(sum(AparicionesTotales))'])\
                .select('Local', col('sum(Por_temporada)').alias('Numero de apariciones totales'), 'Media_victorias')\
                .show()

print('Tiempo total de ejecución: {}'.format(time.time() - t0))

+-----------------+-----------------------------+-------------------+
|            Local|Numero de apariciones totales|    Media_victorias|
+-----------------+-----------------------------+-------------------+
|       AD Almeria|                            4| 0.5138888888888888|
|         Alcorcon|                            5| 0.5533980582524272|
|         Alcoyano|                            1| 0.2857142857142857|
|        Algeciras|                            4| 0.3974358974358974|
|         Alicante|                            1|0.09523809523809523|
|           Alzira|                            1| 0.3684210526315789|
|           Aragon|                            1|0.42105263157894735|
|Atletico Marbella|                            4| 0.4342105263157895|
|           Aviles|                            2|0.34210526315789475|
|        Barakaldo|                            6| 0.6140350877192983|
|     Calvo Sotelo|                            5| 0.5263157894736842|
|        Cartagena| 

Ahora vamos a obtener las medias por Visitante aplicando lo que hemos hecho previamente

In [43]:
#Obtenemos por parte del equipo visitante
t0 = time.time()

mv_filter_vis = vic_mean.select('Visitante', 'Temporada', 'VisWon')\
                                 .withColumn('AparicionesTotales', lit(1))\
                                 .groupBy(['Visitante', 'Temporada'])\
                                 .sum()\
                                 .withColumn('Por_temporada', lit(1))\
                                 .groupBy('Visitante')\
                                 .sum()\
                                 .sort('Visitante')

mv_filter_vis.filter(mv_filter_vis['sum(Por_temporada)'] < 10)\
                .withColumn('Media_victorias', mv_filter_vis['sum(sum(VisWon))'] / mv_filter_vis['sum(sum(AparicionesTotales))'])\
                .select('Visitante', col('sum(Por_temporada)').alias('Numero de apariciones totales'), 'Media_victorias')\
                .show()
            
print('Tiempo total de ejecución: {}'.format(time.time() - t0))

+-----------------+-----------------------------+-------------------+
|        Visitante|Numero de apariciones totales|    Media_victorias|
+-----------------+-----------------------------+-------------------+
|       AD Almeria|                            4|0.09722222222222222|
|         Alcorcon|                            5|               0.29|
|         Alcoyano|                            1|0.14285714285714285|
|        Algeciras|                            4| 0.0641025641025641|
|         Alicante|                            1| 0.2857142857142857|
|           Alzira|                            1|0.10526315789473684|
|           Aragon|                            1|0.05263157894736842|
|Atletico Marbella|                            4|0.14473684210526316|
|           Aviles|                            2|0.15789473684210525|
|        Barakaldo|                            6|0.07017543859649122|
|     Calvo Sotelo|                            5|0.06315789473684211|
|        Cartagena| 

##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

Para esta pregunta vamos a comparar el numero de goles que marca cada visitante en cada partido con el total de goles que ha marcado ese equipo en todo el dataset como visitante.

In [44]:
mean_goals_vis = df_interact.select('GolVisitante', 'Visitante')
mean_goals_vis = mean_goals_vis.groupBy('Visitante')\
            .agg(sum('GolVisitante').alias('Goles_totales'))
        
goles_por_partido = df_interact.select('GolVisitante', 'Visitante')


goles_por_partido.join(mean_goals_vis, 'Visitante')\
                    .withColumn('Media_partido', goles_por_partido.GolVisitante / mean_goals_vis.Goles_totales)\
                    .show()

+---------+------------+-------------+--------------------+
|Visitante|GolVisitante|Goles_totales|       Media_partido|
+---------+------------+-------------+--------------------+
| Logroñes|           1|          307|0.003257328990228013|
| Logroñes|           1|          307|0.003257328990228013|
| Logroñes|           2|          307|0.006514657980456026|
| Logroñes|           1|          307|0.003257328990228013|
| Logroñes|           0|          307|                 0.0|
| Logroñes|           0|          307|                 0.0|
| Logroñes|           0|          307|                 0.0|
| Logroñes|           0|          307|                 0.0|
| Logroñes|           1|          307|0.003257328990228013|
| Logroñes|           2|          307|0.006514657980456026|
| Logroñes|           1|          307|0.003257328990228013|
| Logroñes|           0|          307|                 0.0|
| Logroñes|           1|          307|0.003257328990228013|
| Logroñes|           0|          307|  